## Rating Table to Scorecard
**Authors**: Vinay Wunnava

In this tutorial, you are going to learn how to take a rating table from a `Generalized Additive 2 Model`, also known as `GA2M` and transform it into a scorecard.

**There are a few things that you need for this exercise:**

1. Your DataRobot API Key
2. A trained `Generalized Additive 2 Model` (on any project, with any dataset)

### Import Libraries

In [2]:
import csv
import pandas as pd
import numpy as np
import datarobot as dr

### Connect to DataRobot

In [3]:
dr.Client(token = 'YOUR_API_TOKEN',
          endpoint = 'YOUR_ENDPOINT')

### Define Project and Model ID's.

The project and model ID's can be found in the url (when you use the UI to navigate). Make sure that the GA2M model does not have **any text features as input**.

In [4]:
pid = 'YOUR_PROJECT_ID'
mid = 'YOUR_MODEL_ID'

### Define Functions that download and transform the rating table to scorecard

In [5]:
def download_rating_table(pid, mid):
    """ Download the rating table corresponding to the pid and mid
    """
    project = dr.Project.get(pid)
    rating_tables = rating_tables = project.get_rating_tables()
    rating_table = [rt for rt in rating_tables if rt.model_id == mid][0]
    filepath = './my_rating_table_' + mid + '.csv'
    rating_table.download('./my_rating_table_' + mid + '.csv')
    return filepath

def csv_after_emptylines(filepath, bl_group_n=1, dtype=str):
    """ Read a .CSV into a Pandas DataFrame, but only after at least one blank line has been skipped.
    bl_group_n is the expected number of distinct blocks of blank lines (of any number of rows each) to skip before reading data.
    NB: E.g. pd.read_csv(filepath, skiprows=[0, 1, 2]) works if you know the number of rows to be skipped. Use this function if you have a variable / unknown number of filled rows (to be skipped / ignored) before the empty rows.
    """
    with open(filepath, newline='') as f:
        blank_lines = 0
        bl_groups = 0
        contents = []
        headers = None
        r = csv.reader(f)
        for i, l in enumerate(r):
            if bl_groups < bl_group_n:
                if not l:
                    blank_lines += 1
                    continue
                if blank_lines == 0:
                    continue
                bl_groups += 1
                blank_lines = 0
                headers = l
                continue
            contents.append(l)
        return pd.DataFrame(data=contents, columns=headers, dtype=dtype)

def csv_until_emptyline(filepath, dtype=str):
    """ Read a .CSV into a Pandas DataFrame until a blank line is found, then stop.
    """
    with open(filepath, newline='') as f:
        contents = []
        r = csv.reader(f)
        for i, l in enumerate(r):
            if not l:
                break
            if i == 0:
                headers = l
                continue
            contents.append(l)
        return pd.DataFrame(data=contents)

def extract_intercept(filepath):
    """ Read a .CSV into a Pandas DataFrame until a blank line is found, then stop.
        Extract intercept value and return it
    """ 
    df = csv_until_emptyline(filepath)
    df.rename(columns={df.columns[0]: "raw" }, inplace = True)
    df[['name','value']] = df['raw'].str.split(":",expand=True)
    intercept = pd.to_numeric(df.loc[df.name == 'Intercept','value'].values[0])
    return intercept

def invert_coefficients(intercept, rating_table):
    """ Inverting the sign of intercept and all the coefficients - this is to ensure that the high risk people are given low scores
        Mathematically, we are modelling log of odds and the riskier profiles have high probability
        When we negate the coefficients, it will mean the log of odds of non-risky profiles (- log(p/1-p) = log(1-p/p))
    """
    intercept = - intercept
    rating_table.loc[:,'Coefficient'] = - rating_table['Coefficient'].astype(float)
    return intercept, rating_table

def convert_rating_table_to_scores(intercept, rating_table, min_score=300, max_score=850):
    rating_table['Rel_Coefficient'] = rating_table['Coefficient']
    baseline = intercept
    min_sum_coef = 0
    max_sum_coef = 0
    for feat in rating_table['Feature Name'].unique():
        min_feat_coef = rating_table.loc[rating_table['Feature Name'] == feat]['Coefficient'].min()
        print('Minimum coefficient for feature ' + feat + ' ' + str(min_feat_coef))
        rating_table.loc[rating_table['Feature Name'] == feat,'Rel_Coefficient'] = rating_table['Coefficient'] - min_feat_coef
        baseline += min_feat_coef
        min_sum_coef = min_sum_coef + rating_table.loc[rating_table['Feature Name'] == feat]['Rel_Coefficient'].min()
        max_sum_coef = max_sum_coef + rating_table.loc[rating_table['Feature Name'] == feat]['Rel_Coefficient'].max()

    min_sum_coef = min_sum_coef + baseline
    max_sum_coef = max_sum_coef + baseline
    
    rating_table.loc[:,'Variable Score'] = rating_table['Rel_Coefficient']*((max_score-min_score)/(max_sum_coef - min_sum_coef))
    baseline_score = (((baseline-min_sum_coef)/(max_sum_coef-min_sum_coef))*(max_score-min_score))+min_score
    
    return baseline_score, rating_table.drop(columns=['Coefficient','Rel_Coefficient'])

def get_scorecard(pid,mid, min_score=300, max_score=850):
    """ Download rating table for a particular pid and mid and return scorecard
    """
    filepath = download_rating_table(pid,mid)    
    rating_table_raw = csv_after_emptylines(filepath)
    intercept_raw = extract_intercept(filepath)
    intercept, rating_table = invert_coefficients(intercept_raw, rating_table_raw)
    intercept_score, scorecard = convert_rating_table_to_scores(intercept, rating_table, min_score, max_score)
    
    return intercept_score, scorecard

### Create Scorecard and Intercept Score

In [10]:
intercept_score, scorecard = get_scorecard(pid,mid, min_score=300, max_score=850)

Minimum coefficient for feature addr_state -0.2785427564480061
Minimum coefficient for feature annual_inc -0.45013078726639455
Minimum coefficient for feature delinq_2yrs -0.2776783526503624
Minimum coefficient for feature desc -0.0945789794721339
Minimum coefficient for feature dti -0.04247463282234684
Minimum coefficient for feature earliest_cr_line (Day of Week) -0.48158802883360646
Minimum coefficient for feature earliest_cr_line (Month) -0.046347103102299385
Minimum coefficient for feature earliest_cr_line (Year) -0.037238379052665034
Minimum coefficient for feature emp_length -0.19705440726302845
Minimum coefficient for feature emp_title -0.15997129068918214
Minimum coefficient for feature funded_amnt -0.08483731228497265
Minimum coefficient for feature grade -0.14898665490589103
Minimum coefficient for feature home_ownership -0.006745090976323531
Minimum coefficient for feature inq_last_6mths -0.19540746068248277
Minimum coefficient for feature installment -0.10691166175018307
M

In [11]:
print(intercept_score)
scorecard

300.0


,Feature Name,Feature Strength,Type,Transform1,Value1,Transform2,Value2,Weight,Variable Score
0,addr_state,0.023231943239473876,CAT,One-hot,'AK',,,19.0,0.000000
1,addr_state,0.023231943239473876,CAT,One-hot,'AL',,,75.0,10.868938
2,addr_state,0.023231943239473876,CAT,One-hot,'AR',,,32.0,10.059049
3,addr_state,0.023231943239473876,CAT,One-hot,'AZ',,,137.0,8.894924
4,addr_state,0.023231943239473876,CAT,One-hot,'CA',,,1120.0,11.358706
...,...,...,...,...,...,...,...,...,...
49500,( purpose & term ),0.013551590564601186,2W-INT,One-hot,car,One-hot,36 months,131.0,4.459664
49501,( purpose & term ),0.013551590564601186,2W-INT,One-hot,small_business,One-hot,60 months,86.0,0.000000
49502,( purpose & term ),0.013551590564601186,2W-INT,One-hot,small_business,One-hot,36 months,212.0,12.036993
49503,( purpose & term ),0.013551590564601186,2W-INT,One-hot,major_purchase,One-hot,60 months,60.0,9.711552
